In [ ]:
# Hack to emulate running notebook from root directory.
import os
os.chdir('..')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils import data_handling

In [ ]:
# load data
n_buildings = 1
years = list(range(2012, 2018))
building_ids = [0, 4, 8, 19, 25, 40, 58, 102, 104]

dataset_dir = os.path.join('data','processed')
building_fname_pattern = 'ly_{id}-{year}.csv'

# Load prior scenario samples.
scenarios_path = os.path.join('experiments','results',f'sampled_scenarios_{n_buildings}b.csv')
scenarios,_ = data_handling.load_scenarios(scenarios_path)
n_buildings = scenarios.shape[1]

# Load profiles data.
building_year_pairs = np.unique([tuple([int(v) for v in pair]) for pair in scenarios[:,0,:2]],axis=0)
load_profiles = {
    f'{building_id}-{year}': pd.read_csv(
        os.path.join(dataset_dir, building_fname_pattern.format(id=building_id, year=year)),
        usecols=['Equipment Electric Power [kWh]']
        )['Equipment Electric Power [kWh]'].to_numpy()\
            for (building_id, year) in building_year_pairs
}

In [ ]:
# plot example profiles
year = 2015
for id in building_ids[1:5]:

    fig,ax = plt.subplots(figsize=(15,5))
    plt.plot(load_profiles[f'{id}-{year}'][1*30*24:7*30*24], label=f'{id}-{year}', color='xkcd:cerulean', lw=1)
    plt.axis('off')
    plt.ylim(0)
    plt.margins(0,0)
    plt.tight_layout()
    plt.savefig(os.path.join('plots','graphical_abstract','example_profiles',f'profile_{id}-{year}.png'),bbox_inches='tight',dpi=300)
    plt.show()

In [ ]:
# plot profile construction
year = 2015
for id in building_ids[1:4]:
    fig,ax = plt.subplots(figsize=(15,5))
    profile  = load_profiles[f'{id}-{year}'][3*30*24:7*30*24]
    plt.plot(profile, label=f'{id}-{year}', color='xkcd:cerulean', lw=1)
    plt.hlines(np.mean(profile), 0, len(profile), color='xkcd:dark grey', linestyle='--', lw=2.5)
    plt.annotate("", xy=(24*7, 0), xytext=(24*7, np.mean(profile)), arrowprops=dict(arrowstyle="<->", color='xkcd:dark grey', linewidth=2.5))
    plt.hlines(np.max(profile), 0, len(profile), color='xkcd:dark grey', linestyle=':', lw=2.5)
    plt.annotate("", xy=(24*4.5, 0), xytext=(24*4.5, np.max(profile)), arrowprops=dict(arrowstyle="<->", color='xkcd:dark grey', linewidth=2.5))
    plt.axis('off')
    plt.ylim(0)
    plt.margins(0,0)
    plt.tight_layout()
    plt.savefig(os.path.join('plots','graphical_abstract','example_profiles',f'construction_{id}-{year}.png'),bbox_inches='tight',dpi=300)
    plt.show()

In [ ]:
from scenarioReducer import Fast_forward

In [ ]:
# random scenario reduction plot
n_samples = 30
np.random.seed(1)
xys = np.random.normal(0,1,(n_samples,2))

probs = np.ones(shape=n_samples)/n_samples
FFreducer = Fast_forward(xys.T, probs)
reduced_scenario_stats, reduced_probs, reduced_indices = FFreducer.reduce(distance=1,n_scenarios=5)
reduced_scenario_stats = reduced_scenario_stats.T

fig,ax = plt.subplots(figsize=(8,5))
plt.scatter(xys[:,0],xys[:,1],color='xkcd:dark grey',s=400,alpha=0.75,lw=0)
plt.scatter(reduced_scenario_stats[:,0],reduced_scenario_stats[:,1],color='xkcd:cerulean',s=400,alpha=0.8,lw=0)

plt.xticks([])
plt.yticks([])
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')
for side in ['right','top']:
        ax.spines[side].set_visible(False)
ax.plot((1), (0), ls="", marker=">", ms=8, color="k",
        transform=ax.transAxes, clip_on=False)
ax.plot((0), (1), ls="", marker="^", ms=8, color="k",
        transform=ax.transAxes, clip_on=False)

plt.tight_layout()
plt.savefig(os.path.join('plots','graphical_abstract','scenario_reduction.png'),bbox_inches='tight',dpi=300)
plt.show()